In [1]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [2]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [3]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [4]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

# Define the estimator
estimator = XGBClassifier(tree_method="hist")

# Define hyperparameters for tuning
hyperparameters = {
    "n_estimators": stats.randint(50, 150),  # Increased range
    "learning_rate": stats.uniform(0.05, 0.1),  # Adjusted range
    "colsample_bytree": stats.uniform(0.6, 0.4),
    "colsample_bylevel": stats.uniform(0.6, 0.4),
    "colsample_bynode": stats.uniform(0.6, 0.4),
    "max_depth": stats.randint(15, 50),  # Increased range
    "subsample": stats.uniform(0.6, 0.4),
    "gamma": stats.uniform(0, 2),  # Adjusted range
    "reg_lambda": stats.uniform(0, 2),  # Adjusted range
    "reg_alpha": stats.uniform(0, 2),  # Adjusted range
}

# Define the RandomizedSearchCV parameters
random_search = RandomizedSearchCV(estimator, 
                                   param_distributions=hyperparameters, 
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500, 
                                   cv=5, 
                                   verbose=10, 
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_xgb/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"xgb_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
        

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 1/5; 2/500] START colsample_bylevel=0.6477955519049498, colsample_bynode=0.6191246040055096, colsample_bytree=0.8832874303390754, gamma=0.5257173750746231, learning_rate=0.08425860979032368, max_depth=47, n_estimators=118, reg_alpha=0.8464045034102996, reg_lambda=1.7098423280487807, subsample=0.6424800062794156
[CV 1/5; 2/500] END colsample_bylevel=0.6477955519049498, colsample_bynode=0.6191246040055096, colsample_bytree=0.8832874303390754, gamma=0.5257173750746231, learning_rate=0.08425860979032368, max_depth=47, n_estimators=118, reg_alpha=0.8464045034102996, reg_lambda=1.7098423280487807, subsample=0.6424800062794156;, score=(train=0.967, test=0.842) total time=   3.9s
[CV 1/5; 3/500] START colsample_bylevel=0.889234753552802, colsample_bynode=0.9303059839733908, colsample_bytree=0.762490899741336, gamma=1.344202658658771, learning_rate=0.10149603095502985, max_depth=38, n_estimators=108, reg_alpha=0.106838847605733

[CV 2/5; 1/500] START colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821
[CV 2/5; 1/500] END colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821;, score=(train=0.995, test=0.850) total time=   5.9s
[CV 3/5; 3/500] START colsample_bylevel=0.889234753552802, colsample_bynode=0.9303059839733908, colsample_bytree=0.762490899741336, gamma=1.344202658658771, learning_rate=0.10149603095502985, max_depth=38, n_estimators=108, reg_alpha=0.10683884760573337, reg_lambda=0.7718306225048615, subsample=0.991479380279631

[CV 4/5; 1/500] START colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821
[CV 4/5; 1/500] END colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821;, score=(train=0.996, test=0.865) total time=   6.2s
[CV 1/5; 4/500] START colsample_bylevel=0.7195536325220986, colsample_bynode=0.8718135433164691, colsample_bytree=0.767134144807683, gamma=1.3823608068741715, learning_rate=0.06933529124703727, max_depth=31, n_estimators=122, reg_alpha=0.835474781334701, reg_lambda=0.33913876962090406, subsample=0.85896715418878

[CV 5/5; 1/500] START colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821
[CV 5/5; 1/500] END colsample_bylevel=0.6936519661172889, colsample_bynode=0.7479693786080452, colsample_bytree=0.9807657041802013, gamma=0.11346111587101615, learning_rate=0.11499702039021976, max_depth=49, n_estimators=126, reg_alpha=1.4543244949944933, reg_lambda=1.5467134426066833, subsample=0.7272144777990821;, score=(train=0.996, test=0.860) total time=   6.1s
[CV 4/5; 3/500] START colsample_bylevel=0.889234753552802, colsample_bynode=0.9303059839733908, colsample_bytree=0.762490899741336, gamma=1.344202658658771, learning_rate=0.10149603095502985, max_depth=38, n_estimators=108, reg_alpha=0.10683884760573337, reg_lambda=0.7718306225048615, subsample=0.991479380279631

[CV 2/5; 2/500] START colsample_bylevel=0.6477955519049498, colsample_bynode=0.6191246040055096, colsample_bytree=0.8832874303390754, gamma=0.5257173750746231, learning_rate=0.08425860979032368, max_depth=47, n_estimators=118, reg_alpha=0.8464045034102996, reg_lambda=1.7098423280487807, subsample=0.6424800062794156
[CV 2/5; 2/500] END colsample_bylevel=0.6477955519049498, colsample_bynode=0.6191246040055096, colsample_bytree=0.8832874303390754, gamma=0.5257173750746231, learning_rate=0.08425860979032368, max_depth=47, n_estimators=118, reg_alpha=0.8464045034102996, reg_lambda=1.7098423280487807, subsample=0.6424800062794156;, score=(train=0.967, test=0.838) total time=   3.8s
[CV 5/5; 2/500] START colsample_bylevel=0.6477955519049498, colsample_bynode=0.6191246040055096, colsample_bytree=0.8832874303390754, gamma=0.5257173750746231, learning_rate=0.08425860979032368, max_depth=47, n_estimators=118, reg_alpha=0.8464045034102996, reg_lambda=1.7098423280487807, subsample=0.642480006279415

...

[CV 2/5; 406/500] START colsample_bylevel=0.7844257073812887, colsample_bynode=0.963042472132932, colsample_bytree=0.8846219566078786, gamma=1.737262122261249, learning_rate=0.051879605003801024, max_depth=28, n_estimators=113, reg_alpha=0.07387147127613569, reg_lambda=1.6573083618263038, subsample=0.7820234160476992
[CV 2/5; 406/500] END colsample_bylevel=0.7844257073812887, colsample_bynode=0.963042472132932, colsample_bytree=0.8846219566078786, gamma=1.737262122261249, learning_rate=0.051879605003801024, max_depth=28, n_estimators=113, reg_alpha=0.07387147127613569, reg_lambda=1.6573083618263038, subsample=0.7820234160476992;, score=(train=0.938, test=0.831) total time=   5.8s
[CV 1/5; 408/500] START colsample_bylevel=0.7828442849037748, colsample_bynode=0.7235844117025644, colsample_bytree=0.7963259845426091, gamma=1.2360111523961055, learning_rate=0.05555034532204616, max_depth=42, n_estimators=74, reg_alpha=0.09891103180066918, reg_lambda=1.051518055478039, subsample=0.6723342677

[CV 1/5; 409/500] START colsample_bylevel=0.8907289776802074, colsample_bynode=0.9966694575571593, colsample_bytree=0.9194422521347686, gamma=1.2132636822845866, learning_rate=0.07111846932725273, max_depth=29, n_estimators=78, reg_alpha=1.4688918861191802, reg_lambda=1.0069088084529008, subsample=0.8741455472239719
[CV 1/5; 409/500] END colsample_bylevel=0.8907289776802074, colsample_bynode=0.9966694575571593, colsample_bytree=0.9194422521347686, gamma=1.2132636822845866, learning_rate=0.07111846932725273, max_depth=29, n_estimators=78, reg_alpha=1.4688918861191802, reg_lambda=1.0069088084529008, subsample=0.8741455472239719;, score=(train=0.936, test=0.846) total time=   4.8s
[CV 4/5; 410/500] START colsample_bylevel=0.666708665371854, colsample_bynode=0.8654741222052138, colsample_bytree=0.9708777014127878, gamma=1.4796280932217163, learning_rate=0.12716897872542748, max_depth=43, n_estimators=73, reg_alpha=1.8591797278445998, reg_lambda=0.6118258093538198, subsample=0.7777406163528

[CV 4/5; 416/500] START colsample_bylevel=0.6988732867362375, colsample_bynode=0.9807313033125791, colsample_bytree=0.8820151420108508, gamma=0.3354092689600865, learning_rate=0.1465693928744461, max_depth=33, n_estimators=58, reg_alpha=1.9749646697064986, reg_lambda=0.43859365517940496, subsample=0.8261424701953444
[CV 4/5; 416/500] END colsample_bylevel=0.6988732867362375, colsample_bynode=0.9807313033125791, colsample_bytree=0.8820151420108508, gamma=0.3354092689600865, learning_rate=0.1465693928744461, max_depth=33, n_estimators=58, reg_alpha=1.9749646697064986, reg_lambda=0.43859365517940496, subsample=0.8261424701953444;, score=(train=0.975, test=0.854) total time=   3.9s
[CV 2/5; 418/500] START colsample_bylevel=0.7964328444302354, colsample_bynode=0.9955436509504323, colsample_bytree=0.9870548406547732, gamma=0.7341147719309411, learning_rate=0.13775404596485524, max_depth=32, n_estimators=120, reg_alpha=0.10978239798260248, reg_lambda=1.0839200587708637, subsample=0.8531560576

[CV 1/5; 420/500] START colsample_bylevel=0.6834893124039735, colsample_bynode=0.7651048290668374, colsample_bytree=0.919492243387345, gamma=0.7274883455788137, learning_rate=0.1153356457612956, max_depth=38, n_estimators=143, reg_alpha=1.4599685058794638, reg_lambda=1.3146469252028266, subsample=0.6695466882989651
[CV 1/5; 420/500] END colsample_bylevel=0.6834893124039735, colsample_bynode=0.7651048290668374, colsample_bytree=0.919492243387345, gamma=0.7274883455788137, learning_rate=0.1153356457612956, max_depth=38, n_estimators=143, reg_alpha=1.4599685058794638, reg_lambda=1.3146469252028266, subsample=0.6695466882989651;, score=(train=0.948, test=0.849) total time=   4.8s
[CV 1/5; 421/500] START colsample_bylevel=0.89993854560302, colsample_bynode=0.7494500777475506, colsample_bytree=0.7359695135055797, gamma=1.225277539725215, learning_rate=0.09290095152446634, max_depth=34, n_estimators=67, reg_alpha=1.1580648597708882, reg_lambda=1.940155171884587, subsample=0.7077171584853432
[

[CV 3/5; 429/500] END colsample_bylevel=0.9860238821000077, colsample_bynode=0.9551933835368698, colsample_bytree=0.6596546425413502, gamma=1.158748066181969, learning_rate=0.07924167111126812, max_depth=41, n_estimators=99, reg_alpha=0.40184817057663436, reg_lambda=1.5725296063360168, subsample=0.7410939691066722;, score=(train=0.954, test=0.843) total time=   4.7s
[CV 1/5; 431/500] START colsample_bylevel=0.9467562341047747, colsample_bynode=0.8182506396299617, colsample_bytree=0.7428621842160151, gamma=0.5995818732257179, learning_rate=0.07277448309946082, max_depth=21, n_estimators=110, reg_alpha=0.2860552003229262, reg_lambda=1.4354053275007514, subsample=0.6084222467400351
[CV 1/5; 431/500] END colsample_bylevel=0.9467562341047747, colsample_bynode=0.8182506396299617, colsample_bytree=0.7428621842160151, gamma=0.5995818732257179, learning_rate=0.07277448309946082, max_depth=21, n_estimators=110, reg_alpha=0.2860552003229262, reg_lambda=1.4354053275007514, subsample=0.608422246740

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_xgb/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Test Score")

plt.grid()
plt.xlabel('Sorted Test Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank (top 15)')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train set, Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Test  set, Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(results_path, file_name), "rb"))
model = random_search.best_estimator_

#model = XGBClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance XGB')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In the context of XGBoost, these scores are often computed based on the following:

- Weight: The number of times a feature appears in a tree across the ensemble of trees.
- Gain: The average gain of a feature when it is used in trees.
- Cover: The average coverage of a feature when it is used in trees.

In [ ]:
# TEST
for col in ['param_colsample_bylevel', 'param_colsample_bynode',
       'param_colsample_bytree', 'param_gamma', 'param_learning_rate',
       'param_max_depth', 'param_n_estimators', 'param_reg_alpha',
       'param_reg_lambda', 'param_subsample']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['rank_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['rank_test_score'], m * sorted_cv['rank_test_score'] + b, c='r', label="Regression Line")
    plt.plot(sorted_cv['rank_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Hyperparameter Values')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:
for param in ['param_colsample_bylevel', 'param_colsample_bynode', 'param_colsample_bytree',
                     'param_gamma', 'param_learning_rate', 'param_max_depth', 'param_n_estimators', 
                     'param_reg_alpha', 'param_reg_lambda', 'param_subsample']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


In [ ]:
# IF WE SCALE: SCALE ONLY TRAIN DATA SET
# BOOSTING ALG:

# - Feature Selection: AUTOMATIC (NON CRITICAL)
# - Scaling: AUTOMATIC
# - Handling Missing Values: AUTOMATIC
# - Outliers: NEED TO DEAL WITH